In [1]:
!pip install tensorflow keras imutils

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from tensorflow.keras.preprocessing.image import img_to_array, ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils import class_weight

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import os
from imutils import paths

In [5]:
imagePaths = sorted(list(paths.list_images('/content/drive/MyDrive/ML datasets/rice_leaf_diseases')))
random.seed(42)
random.shuffle(imagePaths)

In [6]:
data = []
labels = []
image_dims = (224, 224, 3)

In [7]:
for imagePath in imagePaths:
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (image_dims[1], image_dims[0]))
    image = img_to_array(image)
    data.append(image)
    l = imagePath.split(os.path.sep)[-2].split("_")
    labels.append(l)

In [8]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [9]:
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)

In [10]:
def MobileNetV2_model(learning_rate, input_shape, class_number):
    baseModel = MobileNetV2(include_top=False, input_tensor=Input(shape=input_shape))
    for layer in baseModel.layers[:-4]:
        layer.trainable = False  # Freeze layers

    model = Sequential()
    model.add(baseModel)
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(50, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(class_number, activation='softmax'))

    return model


In [11]:
bs = 32
lr = 0.0001
epochs = 32 # Increased number of epochs
class_number = len(mlb.classes_)
input_shape = (224, 224, 3)

In [12]:
model = MobileNetV2_model(lr, input_shape, class_number)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

<ipython-input-10-a7bba6b4b957>:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  baseModel = MobileNetV2(include_top=False, input_tensor=Input(shape=input_shape))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [13]:
trainX, testX, trainY, testY = train_test_split(data, labels, test_size=0.20)

In [14]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode="nearest"
)
datagen.fit(trainX)

In [15]:
from tensorflow.keras.callbacks import LearningRateScheduler
def lr_schedule(epoch, lr):
    if epoch > 0 and epoch % 10 == 0:  # Change learning rate every 10 epochs
        return lr * 0.5
    return lr


In [16]:
lr_scheduler = LearningRateScheduler(lr_schedule)

In [17]:
train_labels = trainY.argmax(axis=1)
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(train_labels), y=train_labels)
class_weight_dict = dict(enumerate(class_weights))

In [18]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [22]:
H = model.fit(datagen.flow(trainX, trainY, batch_size=bs),
              steps_per_epoch=len(trainX) // bs,
              validation_data=(testX, testY),
              validation_steps=len(testX) // bs,
              epochs=10,  # Make sure epochs are set here
              class_weight=class_weight_dict
          )
model.export("/content/saved_model")  # Save in SavedModel format

# replace with path where you want to store the model

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - accuracy: 0.9818 - loss: 0.0623 - val_accuracy: 0.8750 - val_loss: 0.4123
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 281ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8750 - val_loss: 0.4123
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.9779 - loss: 0.0795 - val_accuracy: 0.9167 - val_loss: 0.3614
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 433ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9167 - val_loss: 0.3614
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.9661 - loss: 0.1207 - val_accuracy: 0.9167 - val_loss: 0.3254
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 290ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9167 - val_loss: 0.3254
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - accuracy: 0.9609 - loss: 0.0726 - val_accuracy: 0.9167 - val_loss: 0.3937
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 290ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy

In [23]:
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=32)

# for each image in the testing set we need to find the index of the label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,target_names=mlb.classes_))

[INFO] evaluating network...
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
                       precision    recall  f1-score   support

Bacterial leaf blight       0.89      1.00      0.94         8
           Brown spot       0.50      1.00      0.67         4
            Leaf smut       1.00      0.58      0.74        12

             accuracy                           0.79        24
            macro avg       0.80      0.86      0.78        24
         weighted avg       0.88      0.79      0.79        24



In [24]:
import tensorflow as tf

# Convert the SavedModel to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_saved_model("/content/saved_model")
tflite_model = converter.convert()

# Save the TensorFlow Lite model
with open("/content/model.tflite", "wb") as f:
    f.write(tflite_model)